In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from scipy.stats import truncnorm

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import tqdm
import time
import pdb
import argparse
import sys
from shapes_loader import *
from base_model import *
import tf_homo


In [47]:
magic_leap_model = SuperPointNet().cuda()
magic_leap_model.load_state_dict(torch.load('superpoint_v1.pth'))
criterion = nn.Softmax(dim=1) #reduction='elementwise_sum')

clr_imgs = Image.open('../hpatches-dataset/img/images.png').resize((2400,300),Image.ANTIALIAS)
img1 = np.array(clr_imgs)
img1 = img1[:300,:400]
img1 = Image.fromarray(img1).convert('L')
img1 = np.array(img1)

img2 = np.array(clr_imgs)
img2 = img2[:300,400:800]
img2 = Image.fromarray(img2).convert('L')
img2 = np.array(img2)

#plt.imshow(Image.fromarray(img2))

In [48]:
probs_1, desc_1 = magic_leap_model(torch.from_numpy(img1).unsqueeze(0).unsqueeze(1).float().cuda())
probs_2, desc_2 = magic_leap_model(torch.from_numpy(img2).unsqueeze(0).unsqueeze(1).float().cuda())
print(probs_2.shape)
print(desc_2.shape)

torch.Size([1, 65, 37, 50])
torch.Size([1, 256, 37, 50])


In [43]:
# show matching results ...
threshold = 0.4
fig=plt.figure()
plt.imshow()
ipt_sm_1 = criterion(probs_1)
ipt_sm_1 = ipt_sm_1[:,:-1,:,:]
#find the max entry and confidence
idx_conf_1, idx_locs_1 = ipt_sm_1.max(dim=1)
idx_mask_1 = idx_conf_1 > threshold
px = []
py = []
for x in range(prob.shape[2]):
    for y in range(prob.shape[3]):
        if idx_mask_1[0,x,y] == 1:
            #location in the image
            x_ = x*8 +(idx_locs_1[0,x,y]/8)
            y_ = y*8 + (idx_locs_1[0,x,y]%8)
            px.append(x_.item())
            py.append(y_.item())
plt.plot(py,px,'xy')

torch.Size([1, 65, 37, 50])